In [1]:
import pandas as pd

In [4]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
                <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
                </div>"""
    
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

# Concat

In [6]:
s1 = pd.DataFrame(np.arange(0, 3))
s2 = pd.DataFrame(np.arange(5, 8))

display("s1", "s2")

,0
0,0
1,1
2,2
,0
0,5
1,6
2,7


![](https://pandas.pydata.org/docs/_images/08_concat_row.svg)

### названия столбцов одинаковые

In [19]:
pd.concat([s1, s2])

,0
0,0
1,1
2,2
0,5
1,6
2,7


### concat: вертикально и горизонтально 

In [18]:
A = pd.concat([s1, s2], axis=1)
B = pd.concat([s1, s2], axis=0)

display("A", "B")

A
   0  0
0  0  5
1  1  6
2  2  7

B
   0
0  0
1  1
2  2
0  5
1  6
2  7

### названия столбцов разные

In [11]:
df1 = pd.DataFrame(np.arange(0, 9).reshape(3, 3), columns=["a", "b", "c"])
df2 = pd.DataFrame(np.arange(9, 18).reshape(3, 3), columns=["a", "c", "d"])
display("df1", "df2")

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8
,a,c,d
0,9,10,11
1,12,13,14
2,15,16,17


In [14]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0,1.0,2,NaN
1,3,4.0,5,NaN
2,6,7.0,8,NaN
3,9,NaN,10,11.0
4,12,NaN,13,14.0
5,15,NaN,16,17.0


### управление индексом при помощи ключей

In [16]:
c = pd.concat([df1, df2], keys=["df1", "df2"])
c

a    b   c     d
df1 0   0  1.0   2   NaN
    1   3  4.0   5   NaN
    2   6  7.0   8   NaN
df2 0   9  NaN  10  11.0
    1  12  NaN  13  14.0
    2  15  NaN  16  17.0

In [17]:
c.loc["df2"]

,a,b,c,d
0,9,NaN,10,11.0
1,12,NaN,13,14.0
2,15,NaN,16,17.0


# Merge

![](https://pandas.pydata.org/pandas-docs/stable/_images/08_merge_left.svg)

### слияние по 1 общему столбцу

In [20]:
customers = {'CustomerID': [10, 11],
            'Name': ['Mike', 'Marcia'],
            'Address': ['Address for Mike', 'Address for Marcia']}

customers = pd.DataFrame(customers)
customers

,CustomerID,Name,Address
0,10,Mike,Address for Mike
1,11,Marcia,Address for Marcia


In [23]:
orders = {'CustomerID': [10, 11, 10],
          'OrderDate': ["2014-12-01", "2014-12-01", "2014-12-01"]}

orders = pd.DataFrame(orders)
display("customers", "orders")

customers
   CustomerID    Name             Address
0          10    Mike    Address for Mike
1          11  Marcia  Address for Marcia

orders
   CustomerID   OrderDate
0          10  2014-12-01
1          11  2014-12-01
2          10  2014-12-01

In [22]:
customers.merge(orders)

,CustomerID,Name,Address,OrderDate
0,10,Mike,Address for Mike,2014-12-01
1,10,Mike,Address for Mike,2014-12-01
2,11,Marcia,Address for Marcia,2014-12-01


In [24]:
orders.merge(customers)

,CustomerID,OrderDate,Name,Address
0,10,2014-12-01,Mike,Address for Mike
1,10,2014-12-01,Mike,Address for Mike
2,11,2014-12-01,Marcia,Address for Marcia


### слияние по всем общим столбцам

In [25]:
left_data = {'key1': ['a', 'b', 'c'],
             'key2': ['x', 'y', 'z'],
             'lval1': [ 0, 1, 2]}

right_data = {'key1': ['a', 'b', 'c'],
             'key2': ['x', 'a', 'z'],
             'rval1': [ 6, 7, 8 ]}

left = pd.DataFrame(left_data, index=[0, 1, 2])
right = pd.DataFrame(right_data, index=[1, 2, 3])

display("left", "right")

,key1,key2,lval1
0,a,x,0
1,b,y,1
2,c,z,2
,key1,key2,rval1
1,a,x,6
2,b,a,7
3,c,z,8


In [26]:
left.merge(right)

,key1,key2,lval1,rval1
0,a,x,0,6
1,c,z,2,8


### явное задание связующего столбца

In [27]:
left.merge(right, on="key1")

,key1,key2_x,lval1,key2_y,rval1
0,a,x,0,x,6
1,b,y,1,a,7
2,c,z,2,z,8


In [28]:
left.merge(right, on="key2")

,key1_x,key2,lval1,key1_y,rval1
0,a,x,0,a,6
1,c,z,2,c,8


In [30]:
display("left", "right")

,key1,key2,lval1
0,a,x,0
1,b,y,1
2,c,z,2
,key1,key2,rval1
1,a,x,6
2,b,a,7
3,c,z,8


In [29]:
left.merge(right, on=["key1", "key2"])

,key1,key2,lval1,rval1
0,a,x,0,6
1,c,z,2,8


### слияние по индексам

In [31]:
pd.merge(left, right, left_index=True, right_index=True)

,key1_x,key2_x,lval1,key1_y,key2_y,rval1
1,b,y,1,a,x,6
2,c,z,2,b,a,7


### outer, inner, left, right слияние

In [33]:
display("left", "right")

,key1,key2,lval1
0,a,x,0
1,b,y,1
2,c,z,2
,key1,key2,rval1
1,a,x,6
2,b,a,7
3,c,z,8


In [32]:
A = right.merge(left, how="outer")
B = right.merge(left, how="inner")
C = right.merge(left, how="left")
D = right.merge(left, how="right")

display("A", "B", "C", "D")

,key1,key2,rval1,lval1
0,a,x,6.0,0.0
1,b,a,7.0,NaN
2,c,z,8.0,2.0
3,b,y,NaN,1.0
,key1,key2,rval1,lval1
0,a,x,6,0
1,c,z,8,2
,key1,key2,rval1,lval1
0,a,x,6,0.0
1,b,a,7,NaN


# Join

In [34]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], 
     "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)

right = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], 
     "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)

display("left", "right")

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2
,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [35]:
right.join(left)

,C,D,A,B
K0,C0,D0,A0,B0
K2,C2,D2,A2,B2
K3,C3,D3,NaN,NaN


In [36]:
left.join(right)

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


In [38]:
display("left", "right")

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2
,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [39]:
A = left.join(right, how="outer")
B = left.join(right, how="inner")
C = left.join(right, how="right")
D = left.join(right, how="left")

display("A", "B", "C", "D")

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3
,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2
,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


In [40]:
A = left.merge(right, how="outer")
B = left.merge(right, how="inner")
C = left.merge(right, how="right")
D = left.merge(right, how="left")

display("A", "B", "C", "D")

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

### параметры lsuffix и rsuffix

In [41]:
left_data = {'key1': ['a', 'b', 'c'],
             'key2': ['x', 'y', 'z'],
             'lval1': [ 0, 1, 2]}

right_data = {'key1': ['a', 'b', 'c'],
             'key2': ['x', 'a', 'z'],
             'rval1': [ 6, 7, 8 ]}

left = pd.DataFrame(left_data, index=[0, 1, 2])
right = pd.DataFrame(right_data, index=[1, 2, 3])

display("left", "right")

,key1,key2,lval1
0,a,x,0
1,b,y,1
2,c,z,2
,key1,key2,rval1
1,a,x,6
2,b,a,7
3,c,z,8


In [42]:
left.join(right, lsuffix="_l", rsuffix="_r", how="inner")

,key1_l,key2_l,lval1,key1_r,key2_r,rval1
1,b,y,1,a,x,6
2,c,z,2,b,a,7
